# Modeling & Evaluation

Nachdem die Daten einer optimalen Vorbereitung unterzogen wurden, erfolgt nun die Berechnung, Evaluation und Optimierung der Modelle. Hierbei kommen geeignete Modellierungsverfahren zum Einsatz, die den Anforderungen des Forschungsprojekts gerecht werden. Nachdem die Modelle berechnet wurden, erfolgt eine gründliche Evaluation ihrer Leistungen. Hierbei werden verschiedene Evaluationsmetriken herangezogen, um die Vorhersagegenauigkeit, Robustheit und andere relevante Aspekte zu bewerten. Auf Basis der Evaluationsergebnisse werden die Modelle weiter optimiert, um ihre Leistung zu verbessern.

#### Inhalt

- [Topic Modeling](#topic_modeling)
- [Time Series Forecasting](#time_series_forecasting)
- [Exkurs: Sentiment-Analyse zur Identifizierung von Polarisierungsphänomen](#polarization_phenomenon)

---

## Topic Modeling <a name="topic_modeling"></a>

### Modeling

#### 🧩 Modeling ☕

Der folgende Code importiert eigens speziell entwickelte Klassen und Methoden, die für das LDA-Topic-Modeling optimiert sind. Es wird empfohlen, diese Methode zur Erstellung der Modelle zu verwenden. Eine ausführlichere Darstellung und Dokumentation des Codes befindet sich in der Datei `src/models/topic_modeling.py`

In [ ]:
from src.models import topic_modeling as tm
from src.utils import safe_as_pkl
import pandas as pd

# load dataframe
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')

# create & build lda model
lda_model = tm.LdaModel(text=df['preprocessed_text'])
lda_model.build(num_topics=11)

# export
safe_as_pkl(lda_model, path='../models/lda_model.pkl')

#### Ergebnisse visualisieren ☕

In [ ]:
import pyLDAvis.gensim_models
from src.utils import load_pkl

lda_model = load_pkl('../models/lda_model.pkl')

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model.model, lda_model.corpus, lda_model.dictionary, sort_topics=False)
vis

### Evaluation

#### 1. Coherence Score berechnen ☕

Der Coherence Score ist ein Evaluationsmaß für Topic Models, das versucht, die Kohärenz der gefundenen Themen zu bewerten. Die Kohärenz bezieht sich darauf, wie gut die Wörter innerhalb eines Themas zusammenpassen und ob sie eine sinnvolle Bedeutung ergeben. Ein hohes Maß an Kohärenz zeigt an, dass die Themen gut definiert und interpretierbar sind.

In [ ]:
from src.models import topic_modeling as tm
from src.utils import load_pkl

lda_model = load_pkl('../models/lda_model.pkl')
coherence_score = tm.evaluate(model=lda_model.model, text=lda_model.text, dictionary=lda_model.dictionary)

#### 2. Hyperparameter Tuning durchführen 🌒

Hyperparameter Tuning ist ein wichtiger Schritt im Machine Learning, der dazu beiträgt, das bestmögliche Modell zu finden. In diesem Fall wird das bestmögliche Modell anhand des Coherence Scores bemessen. Ziel des Hyperparameter Tuning ist es demnach, das Modell mit dem höchsten Coherence Score zu finden. Folgende Parameter sollen optimiert werden:

`num_topics`, `alpha`, `eta`, `chunksize`, `iterations`, `passes`

*Wertebereiche für Hyperparameter festlegen*

In [ ]:
from hyperopt import hp

search_space = {
    'num_topics': hp.choice('num_topics', [i for i in range(12, 28)]),
    'alpha': hp.choice('alpha', ['symmetric', 'asymmetric', 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
    'eta': hp.choice('eta', ['symmetric', 'auto', 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
    'chunksize':  hp.choice('chunksize', [5000]),
    'iterations': hp.choice('iterations', [50, 100]),
    'passes': hp.choice('passes', [8, 9, 10])
}

*Bayesian Optimization durchführen* ⭕️

Die bayesianische Optimierung ist ein Ansatz zur Suche nach den optimalen Parametereinstellungen eines Modells durch die Kombination von Modellierung der Zielfunktion mittels einer Wahrscheinlichkeitsverteilung und adaptiver Exploration des Suchraums.

*Hinweis: Um die erforderlichen Berechnungen erfolgreich durchzuführen, ist eine leistungsstarke Computerumgebung erforderlich. Die nachfolgenden Berechnungen wurden auf einem virtuellen Server mit den folgenden Spezifikationen durchgeführt*

- *Virtualisierungstechnik: KVM*
- *Prozessor: 18 vCores*
- *RAM: 48GB DDR4 ECC*

In [ ]:
%%script false
from src.models.bayesian_optimization import optimize_topic_modeling

df, optimized_parameters = optimize_topic_modeling('../data/processed/twitter_tweets_processed.feather',search_space, 200)

*Hyperparameter Tuning auswerten*

In [ ]:
import pandas as pd

df = pd.read_feather('../data/modeling/tm_ht_results.feather')
df.sort_values('coherence_score', ascending=False, inplace=True)
df.head(10)

#### 3. Erstellen optimiertes LDA-Modell ⭕️☕

Nach der Ausführung des Hyperparameter-Tunings, kann eine gute Auswahl von Hyperparametern getroffen werden. Mithilfe der optimierten Parameter kann im folgenden das optimierte LDA-Modell erstellt und berechnet werden.

In [ ]:
%%script false
from src.models import topic_modeling as tm
from src.utils import safe_as_pkl
import pandas as pd

# load dataframe
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')

# create & build optimized lda model
lda_model = tm.LdaMulticoreModel(text=df['preprocessed_text'])
lda_model.build(
    
    seed=1688143687, 
    num_topics=20, 
    alpha='asymmetric', 
    eta=0.3,
    chunksize=5000,
    iterations=100,
    passes=10

)

# export
safe_as_pkl(lda_model, path='../models/optimized_lda_model_174.pkl')

#### 4. Ergebnisse visualisieren ☕

In [ ]:
import pyLDAvis.gensim_models
from src.utils import load_pkl

# load optimized lda model
lda_model = load_pkl('../models/optimized_lda_model_174.pkl')

# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model.model, lda_model.corpus, lda_model.dictionary, sort_topics=False)
vis

---

## Time Series Forecasting <a name="time_series_forecasting"></a>

### Data Preparation

Im Rahmen der Berechnung der Time Series Forecasting Modelle ist es notwendig, die Daten einer Transformation zu unterziehen. Dieser Transformationsprozess beinhaltet zunächst die Zuordnung der Tweets zu den zuvor identifizierten Topics. Dadurch entsteht ein neuer Datensatz, der nach den Topics gruppiert wird. Für jedes Topic wird daraufhin eine spezifische Zeitreihe erstellt. Die Erstellung der Zeitreihen erfolgt durch Zählen der Anzahl von Tweets pro Tag, die einem bestimmten Topic zugeordnet werden können. Diese Vorgehensweise ermöglicht es, die zeitliche Entwicklung der Tweets zu analysieren und die relevanten Informationen für die einzelnen Topics zu erfassen.

#### 1. Topic Zuordnung ☕

In [ ]:
%%script false
from src.utils import load_pkl, tweet_topic_assignment
import pandas as pd

lda_model = load_pkl('../models/optimized_lda_model_174.pkl')
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')

df['topics'] = tweet_topic_assignment(lda_model, topic_minimum_probability=0.20)
df.to_feather('../data/modeling/topic_assigned_twitter_tweets.feather')

df.head(5)

#### 2. Topic Time Series erstellen

In [ ]:
from src.models.time_series_forecasting import process_to_timeseries
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

df_topics_assigned = pd.read_feather('../data/modeling/topic_assigned_twitter_tweets.feather')
df_topic_grouped_ts = process_to_timeseries(df_topics_assigned)

# define min max scaler
scaler = MinMaxScaler(feature_range=(0, 100))

list_topic_time_series = []
for topic, df in df_topic_grouped_ts:
    df.drop('topic', axis=1, inplace=True)

    # normalize data and convert the normalized data back into a DataFrame
    normalized_data = scaler.fit_transform(df)
    normalized_df = pd.DataFrame(normalized_data, index=df.index, columns=df.columns)
    
    ttsd = {'id': (topic+1), 'label': None, 'data': normalized_df}

    list_topic_time_series.append(ttsd)

#### 3. Labeling

*Hinweis: Es ist zu beachten, dass alle nicht gelabelten Topics im Datenbestand aussortiert werden und für die weiteren Modellierungsprozesse nicht weiter berücksichtigt werden.*

In [ ]:
topic_labels = {
    4: 'Blockchain-Technologie',
    10: 'Künstliche Intelligenz',
    12: 'VR, AR und Metaverse'
}

In [ ]:
2from src.models.time_series_forecasting import XGBoostModel2
from src.utils import safe_as_pkl

for ttsd in list_topic_time_series:
    if ttsd['id'] in topic_labels:
        xgb_model = XGBoostModel2(id=ttsd['id'], timeseries=ttsd['data'])
        xgb_model.label = topic_labels[ttsd['id']]
        safe_as_pkl(xgb_model, f"../models/xgb_model_{ttsd['id']}.pkl")

#### 4. Aufteilung in Trainings- und Testdaten

In [ ]:
from src.models.time_series_forecasting import XGBoostModel2
from src.utils import load_pkl, safe_as_pkl

xgb_models = [
    load_pkl('../models/xgb_model_4.pkl'),
    load_pkl('../models/xgb_model_10.pkl'),
    load_pkl('../models/xgb_model_12.pkl')
]

for model in xgb_models:
    data_train, data_test = XGBoostModel2.train_test_split(data=model.timeseries, train_size=0.9)
    model.data_train = data_train
    model.data_test = data_test
    safe_as_pkl(model, f"../models/xgb_model_{model.id}.pkl")

### Feature Engineering

Feature Engineering bezeichnet den Prozess der Erzeugung neuer Merkmale oder der Transformation vorhandener Merkmale, um die Leistung von Modellen in maschinellem Lernen zu verbessern. Das Feature Engineering ermöglicht es dem XGBoost-Modell daher, ein umfassenderes Verständnis der Daten zu entwickeln und eine verbesserte Vorhersageleistung zu erzielen. Indem relevante Informationen in den Merkmalen gezielt hervorgehoben oder hinzugefügt werden, können nicht-lineare Zusammenhänge besser erfasst und die Fähigkeit des Modells zur Generalisierung gesteigert werden.

*Hinweis: Das Feature Engineering wird automatisiert bei der Berechnung der Modelle durchgeführt. Folgende Features wurden gewählt:*

```
['day'] = data.index.day
['week'] = data.index.isocalendar().week.astype(int)
['month'] = data.index.month
['weekday'] = data.index.weekday
```

### Modeling

In [ ]:
from src.models.time_series_forecasting import XGBoostModel2
from src.utils import load_pkl, safe_as_pkl

xgb_models = [
    load_pkl('../models/xgb_model_4.pkl'),
    load_pkl('../models/xgb_model_10.pkl'),
    load_pkl('../models/xgb_model_12.pkl')
]

for model in xgb_models:
    model.build(**{'n_estimators': 1000})
    safe_as_pkl(model, f"../models/xgb_model_{model.id}.pkl")

### Evaluation

#### 1. Hyperparameter Tuning durchführen 🎬

Auch für die XGBoost Modelle ist es von Bedeutung ein Hyperparameter Tuning durchzuführen, um die Genauigkeit der Modelle zu verbessern. Hierbei wird das bestmögliche Modell anhand der Evaluationsmetrik des Mean Absolute Error (MAE) beurteilt. Das Ziel des Hyperparameter Tunings besteht darin, jene Modelle zu ermitteln, das den geringsten durchschnittlichen absoluten Fehler aufweist. Folgende Parameter sollen optimiert werden:

`n_estimators`, `learning_rate`, `max_depth`

*Wertebereiche für Hyperparameter festlegen*

In [ ]:
from hyperopt import hp
import numpy as np

search_space = {
    'n_estimators': hp.choice('n_estimators', [100, 500, 1000, 1200, 1400, 1600, 1800, 2000, 4000]),
    'learning_rate': hp.choice('learning_rate', np.arange(0.01, 0.4, 0.01).tolist()), 
    'max_depth': hp.choice('max_depth', [i for i in range(5, 73)])        
}

*Bayesian Optimization durchführen*

In [ ]:
from src.models.time_series_forecasting import XGBoostModel2
from src.models.bayesian_optimization import optimize_xgb_modeling
from src.utils import safe_as_pkl

xgb_models = [
    load_pkl('../models/xgb_model_4.pkl'),
    load_pkl('../models/xgb_model_10.pkl'),
    load_pkl('../models/xgb_model_12.pkl')
]

for model in xgb_models:
    optimized_parameters = optimize_xgb_modeling(model, search_space, 300)
    model.build(**optimized_parameters)
    safe_as_pkl(model, f"../models/xgb_model_{model.id}.pkl")

#### 2. Ergebnisse visualisieren

In [ ]:
from src.models.time_series_forecasting import XGBoostModel2
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt
from src.utils import load_pkl
import pandas as pd

xgb_models = [
    load_pkl('../models/xgb_model_4.pkl'),
    load_pkl('../models/xgb_model_10.pkl'),
    load_pkl('../models/xgb_model_12.pkl')
]

for model in xgb_models:
    data_test_assigned_predictions = model.data_test.assign(predictions=model.predictions)
#     _ = pd.concat([model.data_train, data_test_assigned_predictions])

    plt.rcParams['font.family'] = 'Consolas'
    plt.figure(figsize=(8, 5)) # width, height
    plt.plot(data_test_assigned_predictions['count'], label='data_test', color='#1d3557')
    plt.plot(data_test_assigned_predictions['predictions'], label='predictions', color='#ffb703')
    plt.legend()
    plt.savefig(f'../export/xgb_lchart_{model.id}.svg', format='svg')
    
    avg = model.data_test['count'].mean()
    mae = model.evaluate()
    mape = mean_absolute_percentage_error(model.data_test['count'], data_test_assigned_predictions['predictions'])
    rmse = mean_squared_error(model.data_test['count'], data_test_assigned_predictions['predictions'], squared=False)
    
    print(model.label)
    print(f'Average value of the dependent variable (AVG): {round(avg, 4)}')
    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')
    
    print('\n\n')

---

## Exkurs: Sentiment-Analyse zur Identifizierung von Polarisierungsphänomen <a name="polarization_phenomenon"></a>

### Data Preparation

Im Rahmen des Exkurses wird eine Sentiment-Analyse anhand ausgewählter Twitter-Beiträgen durchgeführt. Hierzu  bedarf es einer Auswahl und Anpassung des zuvor erstellen Datensatzes `topic_assigned_twitter_tweets`. Zunächst wird der Trend und der Zeitraum bestimmt, über den die Sentiment-Analyse der Textdaten durchgeführt werden soll.

In [ ]:
# selection trend and period
topic = 12
start_date, end_date = '2020-08-01', '2021-02-01'

In [ ]:
import pandas as pd

def to_string(preprocessed_text):
    return ' '.join(preprocessed_text)

df_topics_assigned = pd.read_feather('../data/modeling/topic_assigned_twitter_tweets.feather')
df_topics_assigned.dropna(inplace=True)
df_sa = df_topics_assigned[df_topics_assigned['date'].between(start_date, end_date) & df_topics_assigned['topics'].apply(lambda x: (topic-1) in x)]

df_sa = df_sa.copy()
df_sa['preprocessed_text'] = df_sa['preprocessed_text'].apply(to_string)

### Modeling

Im nachfolgenden Code wird der Sentiment-Score für die Textdaten aus dem vorbereiteten Datensatz berechnet. Hierbei wird eine *wörterbuchbasierte* Sentiment-Analyse durchgeführt, um die positive oder negative Tendenz der einzelnen Textnachrichten zu ermitteln. 

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def calculate_sentiment_score(preprocessed_text):
    return analyzer.polarity_scores(preprocessed_text)['compound']

df_sa['sentiment_score'] = df_sa['preprocessed_text'].apply(calculate_sentiment_score)

print(f"Sentiment-Score Topic {topic} ({start_date} - {end_date}): {df_sa['sentiment_score'].mean().round(4)}")

---